In [1]:
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from data_loader import DataLoader
import random
import matplotlib.pyplot as plt
random.seed(1)
np.random.seed(1)

In [ ]:
def split_train_test(images, masks, validation_split=0.8):
    split_index = int(images.shape[0] * 0.8)
    return images[0:split_index], masks[0:split_index], images[split_index:], masks[split_index:]

def normalize(images, masks):
    images = images / 255
    masks = (masks > 0).astype(float)
    return images, masks

In [ ]:
raw_images, raw_masks = DataLoader().get_dataset()
norm_images, norm_masks = normalize(raw_images, raw_masks)

train_img, train_msk, test_img, test_msk = split_train_test(norm_images, norm_masks)

train_img.shape

In [ ]:
valid_data = np.array( (test_img, test_msk) )
valid_data.shape

## HYPERPARAMETERS

In [ ]:
image_size = 512
train_path = 'src/dataset_preview/train/'
EPOCHS = 5
BATCH_SIZE = 2



## Visualization 

In [ ]:
r = 1 #random.randint(0, len(train_img.shape) - 1)
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1,2,1)
ax.imshow(train_img[r], cmap='gray')
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(train_msk[r], cmap='gray')



## Convolutional Blocks

In [ ]:
def down_block(x, filters, kernel_size=(3,3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)

    p = keras.layers.MaxPool2D((2,2), (2,2))(c)
    return c, p 

def up_block(x, skip, filters, kernel_size=(3,3), padding="same", strides=1):
    up_sampling = keras.layers.UpSampling2D((2,2))(x)
    concat = keras.layers.Concatenate()([up_sampling, skip])

    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)

    return c

def bottleneck(x, filters, kernel_size=(3,3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation='relu')(c)

    return c

## UNet Model

In [ ]:
def UNet():
    feature_maps = [64,128,256, 512, 1024]
    inputs = keras.layers.Input( (image_size, image_size, 1) )

    pool_0 = inputs
    conv_1, pool_1 = down_block(pool_0, feature_maps[0]) #512 -> 256
    conv_2, pool_2 = down_block(pool_1, feature_maps[1]) #256 -> 128 
    conv_3, pool_3 = down_block(pool_2, feature_maps[2]) #128 -> 64
    conv_4, pool_4 = down_block(pool_3, feature_maps[3]) #64 -> 32

    bn = bottleneck(pool_4, feature_maps[4])

    ups_1 = up_block(bn, conv_4, feature_maps[3]) #32 -> 64
    ups_2 = up_block(ups_1, conv_3, feature_maps[2]) #64 -> 128
    ups_3 = up_block(ups_2, conv_2, feature_maps[1]) #128 -> 256
    ups_4 = up_block(ups_3, conv_1, feature_maps[0]) #256 -> 512

    outputs = keras.layers.Conv2D(1, (1,1), padding='same', activation='sigmoid')(ups_4)

    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = keras.backend.sum(keras.backend.abs(y_true * y_pred), axis=-1)
    sum_ = keras.backend.sum(keras.backend.square(y_true), axis = -1) + keras.backend.sum(keras.backend.square(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac)


In [ ]:
model = UNet()

model.compile(optimizer='adam', loss=jaccard_distance, metrics=['acc'])
print(model.summary())

In [ ]:
v_data = (test_img, test_msk)
v_data

## Training the model

In [ ]:
train_steps = len(train_img)
test_steps = len(test_img)



model.fit(x=train_img, y=train_msk, batch_size=BATCH_SIZE , epochs=2, validation_data=v_data)

In [ ]:
model.save_weights('UNetW1.h5')

result = model.predict(test_img)



In [ ]:
r = 1

res = (result > 0.175).astype(float)
print(type(result[r]), np.mean(res[r]))

fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1,2,1)
ax.set_title('Ground truth')
ax.imshow(test_msk[r], cmap='gray')


ax1 = fig.add_subplot(1,2,2)
ax1.set_title('Predicted mask')
ax1.imshow(res[r], cmap='gray')


## Test if model works on cpu

In [ ]:
import train_gpu 
import matplotlib.pyplot as plt
import numpy as np

test_train = train.Trainer()
test_train.load_data()
test_train.build_model()
# test_train.train()
# test_train.save()

test_train.model.load_weights('UNetW1.h5')

result = test_train.model.predict(  test_train.validation_data[0]  )



In [ ]:
r = 1
#roc, auc
print(type(result[r]), np.mean(result[r]))

res = (result > 0.175).astype(float)
print(type(result[r]), np.mean(res[r]))

fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1,2,1)
ax.set_title('Ground truth')
ax.imshow(test_train.validation_data[r][1], cmap='gray')


ax1 = fig.add_subplot(1,2,2)
ax1.set_title('Predicted mask')
ax1.imshow(res[r], cmap='gray')

plt.show()

## GPU TEST

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.__version__

In [ ]:
import train as tg

def predict_showcase():
    trainer = tg.Trainer()
    trainer.load_data()
    trainer.build_model()

    trainer.model.load_weights('UNetW1.h5')

    result = trainer.model.predict(trainer.validation_data[0])

    return result

def display(result, threshold=0.25):
    trainer = tg.Trainer()
    trainer.load_data()
    r = 1
    res = (result > threshold).astype(float)
    print(type(result[r]), np.mean(res[r]))

    fig = plt.figure()
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    ax = fig.add_subplot(1,2,1)
    ax.set_title('Ground truth')
    ax.imshow(trainer.validation_data[r][1], cmap='gray')


    ax1 = fig.add_subplot(1,2,2)
    ax1.set_title('Predicted mask')
    ax1.imshow(res[r], cmap='gray')

    plt.show()


result = predict_showcase()




In [ ]:
display(result, 0.175)

In [4]:
from PIL import Image
x = Image.open(f'src/data/raw_img/ISIC_0000000.jpg').convert(mode='L').resize((512,512))
y = Image.open(f'src/data/raw_masks/ISIC_0000000_segmentation.png').convert(mode='L').resize((512,512))

x = np.asarray(x.getdata()).reshape(512,512, -1) 
y = np.asarray(y.getdata()).reshape(512,512, -1)

x = x / 255
y = (y > 0).astype(float)